In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Distributions
ENV["COLUMNS"] = 200;
include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [2]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 3, 30)
end_date   = Date(2020, 6, 30)
travel_threshold_hours = 4.0

pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36
pct_beds_available = 0.25
hospitalized_days = 14

adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours)

11×11 BitMatrix:
 0  1  1  0  1  1  1  1  1  1  0
 1  0  0  1  0  0  1  1  1  0  0
 1  0  0  0  1  1  1  1  0  1  1
 0  1  0  0  0  0  1  1  1  0  0
 1  0  1  0  0  1  0  0  0  1  1
 1  0  1  0  1  0  0  0  0  1  1
 1  1  1  1  0  0  0  1  1  1  0
 1  1  1  1  0  0  1  0  1  1  0
 1  1  0  1  0  0  1  1  0  0  0
 1  0  1  0  1  1  1  1  0  0  0
 0  0  1  0  1  1  0  0  0  0  0

In [3]:
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);

In [4]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [11]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);

In [16]:
N = length(states);
T = (end_date - start_date).value + 1
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days))

11×93 Matrix{Float32}:
   3.70665    7.5852     7.63875   11.3346    15.0291    19.0233    22.6094    29.8746    33.7842   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
   0.0        0.0        0.0        0.0        0.0        3.83893    3.89021    3.85197    3.86398     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
   4.0887     6.0417     7.99425   10.0192    12.1117    14.301     16.3489    20.2464    24.5119      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
   0.0        5.13495    5.301      5.2362     5.1579    10.2483    10.2213    15.8589    20.9335      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
   0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [7]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available)
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours)

11×11 BitMatrix:
 0  1  1  0  1  1  1  1  1  1  0
 1  0  0  1  0  0  1  1  1  0  0
 1  0  0  0  1  1  1  1  0  1  1
 0  1  0  0  0  0  1  1  1  0  0
 1  0  1  0  0  1  0  0  0  1  1
 1  0  1  0  1  0  0  0  0  1  1
 1  1  1  1  0  0  0  1  1  1  0
 1  1  1  1  0  0  1  0  1  1  0
 1  1  0  1  0  0  1  1  0  0  0
 1  0  1  0  1  1  1  1  0  0  0
 0  0  1  0  1  1  0  0  0  0  0

In [8]:
@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurses = n_nurses(states) * Float32(pct_nurses_available);

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [20]:
isolation_spot = vec([7,8])
model = patient_nurse_allocation_new(
    beds,
    vec(forecast_initial),
    forecast_discharged,
    forecast_admitted,
    nurses,
    adj,
    isolation_spot,
    verbose=true
)
sentp = value.(model[:sentnurses])
sentn = value.(model[:sentpatients])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 22.6.0 22G513)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35154 rows, 24552 columns and 3529227 nonzeros
Model fingerprint: 0xdde919f1
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 5e+04]
Presolve removed 29158 rows and 6755 columns
Presolve time: 1.01s
Presolved: 5996 rows, 17797 columns, 3207229 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.55s

Barrier performed 0 iterations in 2.40 seconds (2.48 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      